In [21]:
import pandas as pd 
messages=pd.read_csv('Spam_email_detection_dataset .csv')

In [22]:
messages.shape

(5573, 2)

In [23]:
messages['Category'].value_counts()

Category
ham               4825
spam               747
{"mode":"full"       1
Name: count, dtype: int64

In [24]:
messages = messages[messages['Category'].isin(['ham', 'spam'])]

messages['Category'] = (
    messages['Category'].astype(str).str.strip().str.lower().map({'ham': 0, 'spam': 1}))


In [25]:
messages['Category'].isna().sum()

0

In [26]:
messages['Category'].value_counts()

Category
0    4825
1     747
Name: count, dtype: int64

In [27]:
messages.head()

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [28]:
import nltk
import string
from nltk.corpus import stopwords

In [29]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aaryandeepakjadhav/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/aaryandeepakjadhav/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/aaryandeepakjadhav/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [31]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [32]:
def preprocess_text(text):
    text = text.lower()
    text = ''.join(ch for ch in text if ch not in string.punctuation)
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

In [33]:
messages['Message'] = messages['Message'].apply(preprocess_text)

In [34]:
messages.head()

,Category,Message
0,0,go jurong point crazy available bugis n great ...
1,0,ok lar joking wif u oni
2,1,free entry 2 wkly comp win fa cup final tkts 2...
3,0,u dun say early hor u c already say
4,0,nah dont think go usf life around though


In [128]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1,2)
)


X = tfidf.fit_transform(messages['Message'])
y = messages['Category'].astype(int)


In [129]:
# split again
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# train balanced SVM again
svm_balanced = LinearSVC(class_weight='balanced')
svm_balanced.fit(X_train, y_train)

LinearSVC(class_weight='balanced')

In [130]:
y_train.value_counts(), y_test.value_counts()

(Category
 0    3859
 1     598
 Name: count, dtype: int64,
 Category
 0    966
 1    149
 Name: count, dtype: int64)

In [131]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [132]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = lr.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.9650224215246637
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       966
           1       1.00      0.74      0.85       149

    accuracy                           0.97      1115
   macro avg       0.98      0.87      0.91      1115
weighted avg       0.97      0.97      0.96      1115



In [133]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)

In [134]:
y_pred_rf = rf.predict(X_test)

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

Random Forest Accuracy: 0.9775784753363229
              precision    recall  f1-score   support

           0       0.97      1.00      0.99       966
           1       1.00      0.83      0.91       149

    accuracy                           0.98      1115
   macro avg       0.99      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115



In [135]:
from sklearn.svm import LinearSVC

svm_balanced = LinearSVC(class_weight='balanced')
svm_balanced.fit(X_train, y_train)

LinearSVC(class_weight='balanced')

In [136]:
from sklearn.metrics import classification_report, accuracy_score

y_pred_bal = svm_balanced.predict(X_test)

print("Balanced SVM Accuracy:", accuracy_score(y_test, y_pred_bal))
print(classification_report(y_test, y_pred_bal))

Balanced SVM Accuracy: 0.9829596412556054
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       966
           1       0.95      0.93      0.94       149

    accuracy                           0.98      1115
   macro avg       0.97      0.96      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [137]:
# LINEAR SVM WORKED VERY WELL 
#LOGISTIC REGRESSION THE ACCURACY IS 96.5 
# RANDOM FOREST CLASSIFIER THE ACCURACY IS 97.75 
# SVM THE ACCURACY IS 98.29

In [138]:
# NOW WE WILL DO THE PREDICTION 

In [205]:
phrase_keywords = [
    "click here", "claim", "offer", "limited time",
    "account suspended", "package on hold",
    "address issue", "action required",
    "unusual activity", "verify your"
]

word_keywords = [
    "free", "urgent", "win"
]

import re

suspicious_domains = [
    "amaz0n", "paypa1", "secure-login",
    "verify-account", "free-rewards"
]

def predict_spam(email_text, sender=None):
    text_lower = re.sub(r'\s+', ' ', email_text.lower())

    # 1  Sender domain rule (FIRST)
    if sender:
        sender_lower = sender.lower()
        for dom in suspicious_domains:
            if dom in sender_lower:
                return "SPAM ❌ "

    # 2 Phrase rules
    for phrase in phrase_keywords:
        if phrase in text_lower:
            return "SPAM ❌ )"

    # 3 Word rules
    for word in word_keywords:
        if re.search(rf"\b{word}\b", text_lower):
            return "SPAM ❌ "

    # 4 Money rule
    if re.search(r"(₹|\$|rs\.?|rupees|lakh|crore|\d{4,})", text_lower):
        if "lottery" in text_lower or "win" in text_lower:
            return "SPAM ❌ "

    # 5 ML prediction
    email_text_clean = preprocess_text(email_text)
    email_vector = tfidf.transform([email_text_clean])
    prediction = svm_balanced.predict(email_vector)

    return "SPAM ❌" if prediction[0] == 1 else "NOT SPAM ✅"




In [206]:
predict_spam("Congratulations! You have won a free iPhone. Click here now")

'SPAM ❌ )'

In [207]:
predict_spam("Hi, are we still meeting tomorrow at 10?")

'NOT SPAM ✅'

In [208]:
predict_spam("Congratulations! You’ve been selected to receive a $1000 Amazon gift card. Claim now!")

'SPAM ❌ )'

In [209]:
predict_spam("Hey, are you free for a quick call this evening?")


'SPAM ❌ '

In [210]:
predict_spam("Can you send me the notes for today’s lecture?")

'NOT SPAM ✅'

In [211]:
predict_spam("We noticed unusual activity in your account. Please confirm your identity.")

'SPAM ❌ )'

In [212]:
predict_spam("Win a brand new car today. Click here to participate now.")


'SPAM ❌ )'

In [213]:
predict_spam("Security alert: action required to keep your account active.")

'SPAM ❌ )'

In [214]:
predict_spam("Let’s meet near the library at 5 pm.")

'NOT SPAM ✅'

In [215]:
predict_spam("I’ll be late today, stuck in traffic.")

'NOT SPAM ✅'

In [216]:
predict_spam("hey how are you?")

'NOT SPAM ✅'

In [217]:
predict_spam("You have got a lottery of 1 lakh rupees")

'SPAM ❌ '

In [218]:
predict_spam(
    "Your account has been suspended. Please verify.",sender="support@amaz0n-secure.com")

'SPAM ❌ '

In [219]:
predict_spam( "Hi, are we still meeting tomorrow?",sender="rahul@gmail.com")


'NOT SPAM ✅'

In [220]:
import pickle

# save model
with open("spam_model.pkl", "wb") as f:
    pickle.dump(svm_balanced, f)

# save vectorizer
with open("tfidf.pkl", "wb") as f:
    pickle.dump(tfidf, f)